### Get the weather data for Berlin

In [1]:
from datetime import datetime, timedelta, timezone
from IPython.display import display, JSON
import pandas as pd
import requests
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps

##### Connecting to OWM API

In [2]:
OWM_key = '********************************'

city = "Berlin"
country = "DE"

# achieve the same result with the wather api
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
response
JSON(response.json())

<IPython.core.display.JSON object>

In [3]:
forecast_api = response.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day

weather_info = []

# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city + ',' + country
    weather_info.append(weather_hour)
    
weather_data = pd.DataFrame(weather_info)

In [6]:
# weather_data.head()

In [7]:
# weather_data.info()

### Update data into database using sqlalchemy

#### Establish the connection

In [8]:
import sqlalchemy

In [9]:
schema="name of the schema"
host="provide the aws host"
user="admin"
password="your password for MySQL"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#### Import and update the tables to the SQL

In [10]:
weather_data.assign(datetime = lambda x: pd.to_datetime(x['datetime'])).to_sql('weather', if_exists='append', con=con, index=False)